# Preparing Data
In this notebook, I will prepare a dataframe where for every gameweek, I have taken the data common to every season and then mapped the number of points gained by a player based off their data from the previous gameweek. 
The last gameweek for every season has been removed

## 3rd iteration
looking to add a 'window' of data, i.e. average from all prev gameweeks

## 4th iteration
3 week window of data, removing additional columns based off model feature importances 

## 5th iteration (in progress)
include prv. season stats + col for whether auto-generated \
ohe global_team

In [62]:
import pandas as pd
import os
import numpy as np
import json

pd.set_option("display.max_columns", None) 

In [63]:
# Get list of seasons in the data

data_loc = '../Fantasy-Premier-League/data'

seasons = sorted( [x for x in os.listdir(data_loc) if len(x) == 7], key = lambda y: int(y[:4]))
seasons = seasons[:-1]
print(seasons)
#seasons = ['2023-24']

['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23']


In [64]:
# Dictionary of encodings

encoding_dic ={'2016-17': 'latin-1', 
                '2017-18': 'latin-1', 
                '2018-19': 'latin-1', 
                '2019-20': 'utf-8', 
                '2020-21': 'utf-8', 
                '2021-22': 'utf-8', 
                '2022-23': 'utf-8',
                '2023-24': 'utf-8'} 
#assert [*encoding_dic] == seasons, 'encoding dic not complete'                                              

In [65]:
team_list = pd.read_csv('master_team_list_me.csv')
team_names = pd.Series(team_list.team_name.unique())


def getGlobalTeamID(season, ID):

    '''
    Returns global ID for a given team in a given season
    '''
    
    player_details = team_list.loc[(team_list['team'] == ID) * team_list['season'] == season]

    team = [*player_details['team_name']][0]

    return team_names[team_names == team].index[0]

getGlobalTeamID('2023-24', 1)


0

In [66]:
f = open('table_by_gw.json')
 
# returns JSON object as 
# a dictionary
table_position = json.load(f)


table_position['2017-18']['5']['0']

def get_position(row, season):
    return table_position[season][str(row['round'])][str(getGlobalTeamID(season, row['opponent_team']))]

In [67]:
def get_cum_avg(df, round_, metric):

    temp = df.loc[df['round']  <= round_]

    summy = temp[metric].sum()

    rounds = len(temp['round'])

    return summy/rounds

In [68]:
def get_3w_avg(df, round_, metric):

    temp = df.loc[df['round']  <= round_]
    temp.sort_values('round', inplace =True)

    if len(temp) >= 3:

        temp = temp.tail(3)


    summy = temp[metric].sum()

    rounds = len(temp['round'])

    return round(summy/rounds, 2)

In [69]:
# Need list of columns of each DataFrame for later comparison
# Generate to columns to remove need for both a home/away score and was_home columns
# Need to account for teams changing between seasons
# Generate dictionary of all player data for all seasons

def playerWithData(player, folderPath):

    result = False

    if not player.startswith('.'):
        if 'gw.csv' in os.listdir(folderPath):
            result = True
    
    return result



columns = {}
player_data = {}

for season in seasons:
    print(season)

    players_raw = pd.read_csv(data_loc +'/'+ season + '/players_raw.csv')

    to_keep = ['element_type', 'team', 'id']

    to_merge = players_raw[to_keep].copy()
    to_merge['team_global'] = to_merge.apply(lambda x: getGlobalTeamID(season, x.team), axis = 1)
    #print(to_merge.dtypes)

    for player in os.listdir(data_loc + '/' + season + '/players'):
        if not player.startswith('.'):

            playerFolder = data_loc + '/' + season + '/players/' + player
            if playerWithData(player, playerFolder):
                
                df = pd.read_csv(playerFolder + '/gw.csv', encoding  = encoding_dic[season])

                df['team_scored'] = df.apply(lambda x: x.team_h_score if x.was_home == True else x.team_a_score, axis = 1)
                df['opposition_scored'] = df.apply(lambda x: x.team_h_score if x.was_home == False else x.team_a_score, axis = 1)

                df['GlobalOpponent'] = df.apply(lambda x: getGlobalTeamID(season, x.opponent_team), axis =1)

                for i in ['influence', 'ict_index', 'total_points', 'bps', 'minutes']:
                    
                    df['avg_' + i] = df.apply(lambda x: get_cum_avg(df, x['round'], i), axis = 1)

                for i in ['influence', 'ict_index', 'total_points', 'bps', 'minutes', 'team_scored', 'opposition_scored']:
                    
                    df['avg_3w_' + i] = df.apply(lambda x: get_3w_avg(df, x['round'], i), axis = 1)

                #print(df.columns)
                #display(df['round'])
                #display(df)
                df['OpponentPosition'] = df.apply(lambda x: table_position[season][str(x['round'])][str(getGlobalTeamID(season, x['opponent_team']))], axis = 1)
                #print(col)

                df = df.merge(right = to_merge, how = 'left', left_on='element', right_on='id')

                dic_key = season+player
                player_data[dic_key] = df
                if 'element_type' not in [*df.columns]:
                    print('Oh Dear')
                #print(df.columns)
    columns[season] = df.columns
    #print(df.dtypes)

                

2016-17
2017-18
2018-19
2019-20
2020-21
2021-22
2022-23


In [70]:
#player_data[dic_key]

In [71]:
# Determine which columns to keep

count = 0
for lst in columns.values():

    if count == 0:
        common_columns = [*lst]
        count = 1
    else:
        for column_name in common_columns:
            if column_name not in [*lst]:
                common_columns.remove(column_name)
common_columns
    



['assists',
 'bonus',
 'bps',
 'clean_sheets',
 'creativity',
 'element',
 'fixture',
 'goals_conceded',
 'goals_scored',
 'ict_index',
 'influence',
 'kickoff_time',
 'minutes',
 'opponent_team',
 'own_goals',
 'penalties_missed',
 'penalties_saved',
 'red_cards',
 'round',
 'saves',
 'selected',
 'team_a_score',
 'team_h_score',
 'threat',
 'total_points',
 'transfers_balance',
 'transfers_in',
 'transfers_out',
 'value',
 'was_home',
 'yellow_cards',
 'team_scored',
 'opposition_scored',
 'GlobalOpponent',
 'avg_influence',
 'avg_ict_index',
 'avg_total_points',
 'avg_bps',
 'avg_minutes',
 'avg_3w_influence',
 'avg_3w_ict_index',
 'avg_3w_total_points',
 'avg_3w_bps',
 'avg_3w_minutes',
 'avg_3w_team_scored',
 'avg_3w_opposition_scored',
 'OpponentPosition',
 'element_type',
 'team',
 'team_global']

The variable common_columns now contains a list of columns that every season of data contains and the dictionary player_data contains details for every player, for every week, for every season.

Future data relative to each gameweek has been added in the form of the following week's fixture and whether or not it's at home. 

I will now create a dataframe for each player, for each season containing this data. It will have an additional column, the target, which is the following week's data. All the data will then be joined into a 'master' dataframe. This dataframe has had the final week for each season for each player removed as only the target is needed from that week.

In [72]:
count = 0
for df in player_data.values():

    player_df = df.copy()
    if 'element_type' not in [*player_df.columns]:
        print('Oh Dear')

    # remove columns not in common_columns and other unwanted columns.

    to_drop = [x for x in player_df.columns if x not in common_columns]
    player_df.drop(labels = to_drop, axis = 1, inplace = True)

    to_drop_2 = ['team_a_score', 'team_h_score', 'kickoff_time', 'team', 'opponent_team']
    player_df.drop(labels = to_drop_2, axis = 1, inplace = True)

    # create the target column 
    player_df.sort_values('round', axis =0, inplace = True)

    points = player_df['total_points']
    points.drop(0, axis = 0, inplace = True)
    points_adjusted = points.reset_index(drop = True)

    # create next_fixture and is_home columns
    next_fixtures = player_df['GlobalOpponent']
    next_fixtures.drop(0, axis = 0, inplace = True)
    next_fixtures_adjusted = next_fixtures.reset_index(drop = True)

    next_position = player_df['OpponentPosition']
    next_position.drop(0, axis = 0, inplace = True)
    next_position_adjusted = next_position.reset_index(drop = True)


    is_home = player_df['was_home']
    is_home.drop(0, axis = 0, inplace = True)
    is_home_adjusted = is_home.reset_index(drop = True)

    # remove last gameweek played by a player (accounts for Jan transfer window)
    player_df.drop(player_df.tail(1).index, axis = 0, inplace = True)
    
    # 'Glue' DataFrame back together 
    player_df['next_fixture'] = next_fixtures_adjusted
    player_df['next_position'] = next_position_adjusted
    player_df['is_home'] = is_home_adjusted
    player_df['target'] = points_adjusted


    to_drop_3 = ['was_home']
    player_df.drop(labels = to_drop_3, axis = 1, inplace = True)


    if 'element_type' not in [*player_df.columns]:
        print('Oh Dear')
    
    

    if count == 0:
        master_df = player_df.copy()
        count +=1

        display(master_df)
        
    else:
        count +=1
        df_to_add = player_df.copy()
        master_df = pd.concat([master_df, df_to_add], axis = 0)\
    
master_df.reset_index(inplace=True, drop = True)

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,team_scored,opposition_scored,GlobalOpponent,avg_influence,avg_ict_index,avg_total_points,avg_bps,avg_minutes,avg_3w_influence,avg_3w_ict_index,avg_3w_total_points,avg_3w_bps,avg_3w_minutes,avg_3w_team_scored,avg_3w_opposition_scored,OpponentPosition,element_type,team_global,next_fixture,next_position,is_home,target
0,0,0,0,0,0.0,404,3,0,0,0.0,0.0,0,0,0,0,0,1,0,1844,0.0,0,0,0,0,60,0,1,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,1.00,9,4,16,4,18,True,0
1,0,0,0,0,0.0,404,16,0,0,0.0,0.0,0,0,0,0,0,2,0,1967,0.0,0,-323,66,389,60,0,1,0,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.50,18,4,16,8,11,True,0
2,0,0,0,0,0.0,404,27,0,0,0.0,0.0,0,0,0,0,0,3,0,1849,0.0,0,-299,85,384,59,0,1,1,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.67,11,4,16,13,20,False,0
3,0,0,0,0,0.0,404,37,0,0,0.0,0.0,0,0,0,0,0,4,0,1564,0.0,0,-348,33,381,59,0,4,0,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.33,20,4,16,14,19,True,0
4,0,0,0,0,0.0,404,49,0,0,0.0,0.0,0,0,0,0,0,5,0,1431,0.0,0,-150,19,169,59,0,1,0,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.00,0.33,19,4,16,11,16,False,0
5,0,0,0,0,0.0,404,55,0,0,0.0,0.0,0,0,0,0,0,6,0,1361,0.0,0,-104,24,128,59,0,2,1,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.33,0.33,16,4,16,9,1,True,0
6,0,0,0,0,0.0,404,68,0,0,0.0,0.0,0,0,0,0,0,7,0,1341,0.0,0,-53,29,82,59,0,2,0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.67,0.33,1,4,16,18,12,False,0
7,0,0,0,0,0.0,404,80,0,0,0.0,0.0,0,0,0,0,0,8,0,1333,0.0,0,-48,23,71,59,0,1,1,18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.67,0.67,12,4,16,1,10,False,0
8,0,0,0,0,0.0,404,82,0,0,0.0,0.0,0,0,0,0,0,9,0,1350,0.0,0,-18,11,29,59,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.00,0.33,10,4,16,7,11,True,0
9,0,0,0,0,0.0,404,98,0,0,0.0,0.0,0,0,0,0,0,10,0,1344,0.0,0,-29,9,38,59,0,1,1,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.67,0.67,11,4,16,0,4,False,0


In [73]:
master_df.columns

Index(['assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'fixture', 'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'minutes', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'yellow_cards', 'team_scored', 'opposition_scored', 'GlobalOpponent',
       'avg_influence', 'avg_ict_index', 'avg_total_points', 'avg_bps',
       'avg_minutes', 'avg_3w_influence', 'avg_3w_ict_index',
       'avg_3w_total_points', 'avg_3w_bps', 'avg_3w_minutes',
       'avg_3w_team_scored', 'avg_3w_opposition_scored', 'OpponentPosition',
       'element_type', 'team_global', 'next_fixture', 'next_position',
       'is_home', 'target'],
      dtype='object')

In [74]:
master_df.describe()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,team_scored,opposition_scored,GlobalOpponent,avg_influence,avg_ict_index,avg_total_points,avg_bps,avg_minutes,avg_3w_influence,avg_3w_ict_index,avg_3w_total_points,avg_3w_bps,avg_3w_minutes,avg_3w_team_scored,avg_3w_opposition_scored,OpponentPosition,element_type,team_global,next_fixture,next_position,target
count,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,1.619070e+05,161907.000000,161907.000000,1.619070e+05,1.619070e+05,1.619070e+05,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000,161907.000000
mean,0.038504,0.102547,5.860877,0.102262,4.673113,316.272125,192.892593,0.484210,0.042500,1.686191,7.021421,31.554763,0.001519,0.000939,0.000679,0.001779,20.351165,0.093473,1.730546e+05,5.178584,1.317614,1.137482e+03,1.200112e+04,1.086291e+04,51.184112,0.053259,1.365661,1.382794,12.928965,7.211334,1.737220,1.347720,6.001617,32.308360,7.036522,1.687727,1.320949,5.876438,31.618687,1.367843,1.385829,10.478207,2.578474,13.077755,12.922554,10.476564,1.314254
std,0.208671,0.478746,9.659570,0.302994,10.638103,185.993713,106.293177,0.958564,0.225275,3.008161,12.703084,40.307013,0.039108,0.030626,0.026526,0.042138,11.191257,0.623371,4.635634e+05,13.241490,2.476756,5.631219e+04,4.909296e+04,4.164227e+04,12.331566,0.224550,1.266851,1.274729,8.817238,8.341356,2.144341,1.546833,6.598180,32.119593,10.029500,2.472942,1.902160,7.834448,36.459811,0.824066,0.798195,5.773596,0.855269,8.860109,8.813895,5.772043,2.476339
min,0.000000,0.000000,-21.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,0.000000,-7.000000,-2.180978e+06,0.000000e+00,0.000000e+00,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,-8.500000,0.000000,0.000000,0.000000,-3.000000,-8.500000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,-7.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,157.000000,102.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.000000,0.000000,5.188000e+03,0.000000,0.000000,-1.207000e+03,3.900000e+01,1.170000e+02,44.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.670000,0.670000,5.000000,2.000000,5.000000,5.000000,5.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,313.000000,197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000,2.348000e+04,0.000000,0.000000,-6.400000e+01,3.410000e+02,9.080000e+02,48.000000,0.000000,1.000000,1.000000,12.000000,4.145455,1.025000,0.869565,3.729730,23.181818,0.930000,0.330000,0.330000,1.330000,8.670000,1.330000,1.330000,10.000000,3.000000,12.000000,12.000000,10.000000,0.000000
75%,0.000000,0.000000,10.000000,0.000000,2.400000,470.000000,285.000000,1.000000,0.000000,2.400000,10.400000,90.000000,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,1.212635e+05,2.000000,2.000000,4.200000e+01,3.831000e+03,6.148000e+03,54.000000,0.000000,2.000000,2.000000,20.000000,12.500000,2.716667,2.235294,10.500000,60.793611,12.530000,2.700000,2.000000,11.000000,66.670000,2.000000,2.000000,15.000000,3.000000,20.000000,20.000000,15.000000,2.000000
max,4.000000,3.000000

## Downsample

In [75]:
if len(seasons) > 1:
    to_downsample = master_df.loc[master_df['target'] == 0]
    master_df = master_df.loc[master_df['target']!= 0]

    to_downsample = to_downsample.sample(n = 20000)

    master_df = pd.concat([master_df, to_downsample], axis = 0)



### Save File

In [76]:
master_df.head()

,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,threat,total_points,transfers_balance,transfers_in,transfers_out,value,yellow_cards,team_scored,opposition_scored,GlobalOpponent,avg_influence,avg_ict_index,avg_total_points,avg_bps,avg_minutes,avg_3w_influence,avg_3w_ict_index,avg_3w_total_points,avg_3w_bps,avg_3w_minutes,avg_3w_team_scored,avg_3w_opposition_scored,OpponentPosition,element_type,team_global,next_fixture,next_position,is_home,target
37,1,0,12,0,10.6,51,9,0,0,2.9,18.2,24,0,0,0,0,1,0,145930,0.0,4,0,0,0,45,0,1,3,10,18.200000,2.900000,4.000000,12.000000,24.000000,18.20,2.90,4.00,12.00,24.00,1.0,3.00,1,4,1,19,10,False,1
38,0,0,3,0,0.8,51,20,1,0,0.4,3.4,16,0,0,0,0,2,0,150680,0.0,1,-4880,6903,11783,45,0,0,1,19,10.800000,1.650000,2.500000,7.500000,20.000000,10.80,1.65,2.50,7.50,20.00,0.5,2.00,10,4,1,4,17,False,1
112,0,0,0,0,0.0,424,17,0,0,0.0,0.0,0,0,0,0,0,2,0,2376,0.0,0,-730,23,753,59,0,1,2,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,1.0,1.50,4,3,17,0,8,True,1
113,0,0,5,0,15.2,424,28,0,0,4.6,9.8,14,0,0,0,0,3,0,1957,21.0,1,-481,56,537,59,0,1,3,0,3.266667,1.533333,0.333333,1.666667,4.666667,3.27,1.53,0.33,1.67,4.67,1.0,2.00,8,3,17,19,17,False,1
114,0,0,6,0,11.3,424,38,0,0,4.2,9.2,22,0,0,0,0,4,0,1907,21.0,1,-144,143,287,59,0,4,2,19,4.750000,2.200000,0.500000,2.750000,9.000000,6.33,2.93,0.67,3.67,12.00,2.0,2.33,17,3,17,10,7,True,1


In [77]:
master_df.to_csv('cleaned_data_4.csv')

In [78]:
len(master_df)

88622